# Generación de datos de entrada y funciones de filtrado

In [ ]:
import sys
sys.path.append('../../')
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import lfilter, firwin
from scipy.fftpack import fft,fftshift
from scipy import signal
from tool._fixedInt import *
from random import *
import sounddevice as sd
import soundfile as sf

In [ ]:
## Resolucion de datos
NB_INT  = 32
NB_FRAC = 16

M       = 3

## Set de microfonos
mic_set = 'set1'

## Señal de interés
mic1_path = '../../audios/'+mic_set+'/signal_plus_noise.wav'
mic1, fs   = sf.read(mic1_path)

## Fuente de ruido
mic2_path = '../../audios/'+mic_set+'/noise_only.wav'
mic2, fs = sf.read(mic2_path)

if(len(mic1)>len(mic2)):
    T = len(mic2)-1
else:
    T = len(mic1)-1

Ts      = 1/fs
t       = np.arange(0,T/fs, Ts)
N       = len(t)
n       = np.arange(N)

## Generación de ruido extra
std_w = np.sqrt(0.5)
w     = std_w * np.random.randn(N)

## Normalizando señal y agregando ruido extra
d   = (mic1[:T]+w)/max(mic1[:T]+w)
x   = (mic2[:T]+w)/max(mic2[:T]+w)

# Datos utilizados en el calculo de los coeficientes
d_aux  = arrayFixedInt(NB_INT, NB_FRAC, d)
x_aux = arrayFixedInt(NB_INT, NB_FRAC, x)

d_fp    = np.array([i.fValue for i in d_aux])
x_fp    = np.array([i.fValue for i in x_aux])

## FIR adaptivo
La función a implementar debe responder al siguiente RTL:

<img src="../../doc/sat_trunc_fir.png">

In [ ]:
def adaptive_fir(x,d,h):
    e               = DeFixedInt(NB_INT, NB_FRAC)
    partial_prod    = [DeFixedInt(NB_INT,NB_FRAC), DeFixedInt(NB_INT,NB_FRAC), DeFixedInt(NB_INT+1,NB_FRAC)]
    partial_sum     = [DeFixedInt(NB_INT+1,NB_FRAC), DeFixedInt(NB_INT,NB_FRAC)]

    partial_prod[0].value   = x[0] * h[0]
    partial_prod[1].value   = x[1] * h[1]
    partial_prod[2].value   = x[2] * h[2]

    partial_sum[0].value    = partial_prod[0].fValue + partial_prod[1].fValue
    partial_sum[1].value    = partial_prod[2].fValue + partial_sum[0].fValue
    
    e.value               = d - partial_sum[1].fValue

    return e.fValue

##  Algoritmo LMS
Ahora, la función que represente las operaciones realizadas por el algoritmo LMS deberá seguir al siguiente RTL:

<img src="../../doc/sat_trunc_lms.png">

In [ ]:
def lms(x, h, e):
    h_out = np.zeros(M)
    
    u               = DeFixedInt(NB_INT, NB_FRAC)
    u.value         = 1.5
    uxe             = DeFixedInt(NB_INT, NB_FRAC)
    uxe.value       = u.fValue * e

    partial_prod    = [DeFixedInt(NB_INT, NB_FRAC), DeFixedInt(NB_INT, NB_FRAC), DeFixedInt(NB_INT, NB_FRAC)]
    partial_sum     = [DeFixedInt(NB_INT, NB_FRAC), DeFixedInt(NB_INT, NB_FRAC), DeFixedInt(NB_INT, NB_FRAC)]

    partial_prod[0].value   = uxe.fValue * x[0]
    partial_prod[1].value   = uxe.fValue * x[1]
    partial_prod[2].value   = uxe.fValue * x[2]

    partial_sum[0].value    = partial_prod[0].fValue + h[0]
    partial_sum[1].value    = partial_prod[1].fValue + h[1]
    partial_sum[2].value    = partial_prod[2].fValue + h[2]

    h_out[0]                = partial_sum[0].fValue
    h_out[1]                = partial_sum[1].fValue
    h_out[2]                = partial_sum[2].fValue

    return h_out

## Filtrado de señal con las funciones previas

In [ ]:
X       = np.zeros([M,1])
H       = np.zeros([M,N])
e     = []

iteraciones = len(x_fp) - 1
for i in range(iteraciones):
    X[1:, 0] = X[:-1, 0]
    X[0, 0]  = x_fp[i]
    error = adaptive_fir(x=X[:,0], h=H[:,i], d=d[i])
    e.append(error)

    H[:,i+1] = lms(x=X[:,0], h=H[:,i], e=error)




In [ ]:
# =============================== SALIDAS ===============================
plt.figure(figsize=(15,3))
plt.plot(n[:-1], e)
plt.xlim([0, N])
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Salida del sistema", fontsize=15)
plt.grid(True)

# =============================== COEFICIENTES ===============================
colores = ['b', 'r', 'k']
plt.figure(figsize=(15,3))
for i in range(0,M):
    plt.plot(n,H[i,:], linewidth=2, label=f"$h_{i}$")
plt.title("Coeficientes", fontsize=15, fontweight="bold")
plt.xlabel("n", fontsize=15)
plt.ylabel("h(n)", fontsize=15)
plt.xlim([0, 1000])
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(fontsize=15)
plt.grid(True)
plt.show()

# Amplifico la señal para reproducirla
amp     = 3
d_amp   = [amp*i for i in d]
e_amp   = [amp*i for i in e]

# Reproduzco señal amplificada
print(f"Reproduciendo señal ORIGINAL amplificada x{amp}...")
sd.play(d_amp, fs)
sd.wait()

print(f"Reproduciendo señal FILTRADA amplificada x{amp}...")
sd.play(e_amp, fs)
sd.wait()

In [ ]:
## Guardando señal para testeo
err_hex     = arrayFixedInt(NB_INT, NB_FRAC, e)
mic1_hex    = arrayFixedInt(NB_INT, NB_FRAC, d_fp)
mic2_hex    = arrayFixedInt(NB_INT, NB_FRAC, x_fp)

NB_DEPTH = 15

data = open("out.mem", "w")
for i,v in enumerate(err_hex):
    if(i<2**NB_DEPTH-1):
        data.write(f"0x{v.__hex__()}\n")
data.close()

data = open("mic1.mem", "w")
for i,v in enumerate(mic1_hex):
    if(i<2**NB_DEPTH-1):
        data.write(f"0x{v.__hex__()}\n")
data.close()

data = open("mic2.mem", "w")
for i,v in enumerate(mic2_hex):
    if(i<2**NB_DEPTH-1):
        data.write(f"0x{v.__hex__()}\n")
data.close()